In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s : %(message)s')

In [3]:
# ----load data-----
def load_data(filepath: str):    
    try:
        df = pd.read_csv(filepath)
        print(f"Data loaded successfully: {df.shape[0]} rows, {df.shape[1]} columns.")
        return df
    except FileNotFoundError:
        print('File Not Found! Please check filepath and try again!')
        raise


In [4]:
# ----- Basic Cleaning --------
def str_2_numeric_summary(df: pd.DataFrame,columns:str):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)
            logging.info(f"Column '{col}' cleaned and converted to numeric.")
    return df[col]

In [5]:
# ------ Drop Columns --------
def drop_cols(df:pd.DataFrame, columns: str):
    df.drop(columns=columns, inplace=True)
    logging.info('Columns successfully dropped')
    return df

In [6]:
def one_hot(df: pd.DataFrame, columns: str):
        return pd.get_dummies(df, columns=columns, dtype=int)


In [7]:
# ----------dataset overview-----
def dataset_overview(df: pd.DataFrame):
    logging.info(f'Number of observations : {df.shape[0]}')
    logging.info(f'Number of features : {df.shape[1]}')
    overview = pd.DataFrame({
        "Dtype": df.dtypes,
        "Non-Null Count": df.count(),
        "Null Count": df.isnull().sum(),
        "Unique Values": df.nunique()
    })
    display(overview.head(20))
    return df.describe(include='all')

In [8]:
# -------duplicate data---------
def duplicates(df: pd.DataFrame):
    duplicates = df[df.duplicated()]
    logging.info(f'Number of duplicated rows : {len(duplicates)}')
    if len(duplicates) == 0:
        logging.info(f'No duplicates found')
    return duplicates

In [9]:
# -----missing data---------
def missing_data(df: pd.DataFrame):
    missing_values = df.isnull().sum()
    missing_pct = (missing_values / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Values' : missing_values,
        'Missing Pct' : missing_pct.round(2)
    }).sort_values(by='Missing Pct',ascending=False)
    logging.info(f'---------Missing Data(Top 10)----------\n')
    display(missing_data.head(10))
    return missing_data

In [10]:
# ---column summaries--------
def column_summaries(df: pd.DataFrame):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for i,col in enumerate(numeric_cols,1):
        logging.info(f'{i:<2}. {col:<17} -Min : {df[col].min():<4} -Max : {df[col].max()}')

    categorical_cols = df.select_dtypes(exclude=[np.number]).columns
    for i,col in enumerate(categorical_cols,1):
        uniques = df[col].unique()
        logging.info(f'{i}. {col} | Unique : {df[col].nunique()} | Examples : {uniques[:5]}')
    return numeric_cols, categorical_cols

In [11]:
# outlier detection using IQR
def check_outliers(df: pd.DataFrame, col: str):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outlier, upper_bound, lower_bound

def outlier_summary(df: pd.DataFrame, numeric_cols):
    results = []
    logging.info(f"\n Outlier Summary (IQR Method):")
    for i, col in enumerate(numeric_cols, 1):
        outlier, lower, upper = check_outliers(df, col)
        results.append({
            'column' : col,
            'Outlier_count' : len(outlier),
            'lower_bound' : lower,
            'upper_bound' : upper
        })
    summary_df = pd.DataFrame(results)
    display(summary_df)
    return summary_df
        

In [12]:
# ---- Save Reports ----
import os
def save_summary(df: pd.DataFrame, name: str):
    os.makedirs("eda_reports", exist_ok=True)
    path = f"eda_reports/{name}.csv"
    df.to_csv(path, index=False)
    logging.info(f"Saved report: {path}")

In [13]:
def run_basic_eda(filepath: str):
    df = load_data(filepath)
    columns = ['Hits','Height','Weight','Value','Wage','Release Clause','W/F','SM','IR']
    str_numeric = str_2_numeric_summary(df,columns)
    columns = ['Name','photoUrl','playerUrl',]
    dropped = drop_cols(df, columns)
    cols = ['Preferred Foot', 'A/W', 'D/W']
    onehot = one_hot(df,cols)
    overview = dataset_overview(df)
    duplicate = duplicates(df)
    missing = missing_data(df)
    numeric_cols, category_cols = column_summaries(df)
    outlier_df = outlier_summary(df, numeric_cols)
    
    save_summary(missing, "missing_data")
    save_summary(outlier_df, "outlier_summary")

    return {
        'data': df,
        'str_numeric' : str_numeric,
        'dropped' : dropped,
        'onehot' : onehot,
        'overview': overview,
        'duplicate': duplicate,
        'missing': missing,
        'outliers': outlier_df,
        'numeric_cols': numeric_cols,
        'category_cols': category_cols
    }


In [14]:
if __name__ == '__main__':
    df = run_basic_eda("data/fifa21 raw data v2.csv")

C:\Users\Maxwe\AppData\Local\Temp\ipykernel_18420\3478159876.py:4: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
INFO : Column 'Hits' cleaned and converted to numeric.
INFO : Column 'Height' cleaned and converted to numeric.
INFO : Column 'Weight' cleaned and converted to numeric.
INFO : Column 'Value' cleaned and converted to numeric.


Data loaded successfully: 18979 rows, 77 columns.


INFO : Column 'Wage' cleaned and converted to numeric.
INFO : Column 'Release Clause' cleaned and converted to numeric.
INFO : Column 'W/F' cleaned and converted to numeric.
INFO : Column 'SM' cleaned and converted to numeric.
INFO : Column 'IR' cleaned and converted to numeric.
INFO : Columns successfully dropped
INFO : Number of observations : 18979
INFO : Number of features : 74


,Dtype,Non-Null Count,Null Count,Unique Values
ID,int64,18979,0,18979
LongName,object,18979,0,18852
Nationality,object,18979,0,164
Age,int64,18979,0,29
↓OVA,int64,18979,0,47
POT,int64,18979,0,48
Club,object,18979,0,682
Contract,object,18979,0,131
Positions,object,18979,0,640
Height,float64,18979,0,52


INFO : Number of duplicated rows : 0
INFO : No duplicates found
INFO : ---------Missing Data(Top 10)----------



,Missing Values,Missing Pct
Loan Date End,17966,94.66
Hits,2595,13.67
Nationality,0,0.00
Age,0,0.00
↓OVA,0,0.00
POT,0,0.00
Club,0,0.00
Contract,0,0.00
ID,0,0.00
LongName,0,0.00


INFO : 1 . ID                -Min : 41   -Max : 259216
INFO : 2 . Age               -Min : 16   -Max : 53
INFO : 3 . ↓OVA              -Min : 47   -Max : 93
INFO : 4 . POT               -Min : 47   -Max : 95
INFO : 5 . Height            -Min : 5.0  -Max : 206.0
INFO : 6 . Weight            -Min : 50.0 -Max : 203.0
INFO : 7 . BOV               -Min : 48   -Max : 93
INFO : 8 . Value             -Min : 0.0  -Max : 975.0
INFO : 9 . Wage              -Min : 0.0  -Max : 950.0
INFO : 10. Release Clause    -Min : 0.0  -Max : 999.0
INFO : 11. Attacking         -Min : 42   -Max : 437
INFO : 12. Crossing          -Min : 6    -Max : 94
INFO : 13. Finishing         -Min : 3    -Max : 95
INFO : 14. Heading Accuracy  -Min : 5    -Max : 93
INFO : 15. Short Passing     -Min : 7    -Max : 94
INFO : 16. Volleys           -Min : 3    -Max : 90
INFO : 17. Skill             -Min : 40   -Max : 470
INFO : 18. Dribbling         -Min : 5    -Max : 96
INFO : 19. Curve             -Min : 4    -Max : 94
INFO : 20.

,column,Outlier_count,lower_bound,upper_bound
0,ID,250,302103.75,154953.75
1,Age,8,41.00,9.00
2,↓OVA,156,83.50,47.50
3,POT,153,87.00,55.00
4,Height,62,201.00,161.00
...,...,...,...,...
58,PAS,181,83.50,31.50
59,DRI,441,87.00,39.00
60,DEF,0,105.00,-7.00
61,PHY,117,90.50,38.50


INFO : Saved report: eda_reports/missing_data.csv
INFO : Saved report: eda_reports/outlier_summary.csv


In [15]:
df['data']

,ID,LongName,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,Lionel Messi,Argentina,33,93,93,\r\n\r\n\r\n\r\nFC Barcelona,2004 ~ 2021,"RW, ST, CF",170.0,...,Medium,Low,5.0,85,92,91,95,38,65,771.0
1,20801,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,\r\n\r\n\r\n\r\nJuventus,2018 ~ 2022,"ST, LW",187.0,...,High,Low,5.0,89,93,81,89,35,77,562.0
2,200389,Jan Oblak,Slovenia,27,91,93,\r\n\r\n\r\n\r\nAtlético Madrid,2014 ~ 2023,GK,188.0,...,Medium,Medium,3.0,87,92,78,90,52,90,150.0
3,192985,Kevin De Bruyne,Belgium,29,91,91,\r\n\r\n\r\n\r\nManchester City,2015 ~ 2023,"CAM, CM",181.0,...,High,High,4.0,76,86,93,88,64,78,207.0
4,190871,Neymar da Silva Santos Jr.,Brazil,28,91,91,\r\n\r\n\r\n\r\nParis Saint-Germain,2017 ~ 2022,"LW, CAM",175.0,...,High,Medium,5.0,91,85,86,94,36,59,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,247223,Ao Xia,China PR,21,47,55,\r\n\r\n\r\n\r\nWuhan Zall,2018 ~ 2022,CB,178.0,...,Medium,Medium,1.0,64,28,26,38,48,51,NaN
18975,258760,Ben Hough,England,17,47,67,\r\n\r\n\r\n\r\nOldham Athletic,2020 ~ 2021,CM,175.0,...,Medium,Medium,1.0,64,40,48,49,35,45,NaN
18976,252757,Ronan McKinley,England,18,47,65,\r\n\r\n\r\n\r\nDerry City,2019 ~ 2020,CM,179.0,...,Medium,Medium,1.0,63,39,44,46,40,53,NaN
18977,243790,Zhen'ao Wang,China PR,20,47,57,\r\n\r\n\r\n\r\nDalian YiFang FC,2020 ~ 2022,RW,175.0,...,Medium,Medium,1.0,58,49,41,49,30,44,NaN
